In [3]:
#自定义层
#CenteredLayer类通过继承Module类自定义了一个将输入减掉均值后输出的层，
#并将层的计算定义在了forward函数里。这个层里不含模型参数。
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super(CenteredLayer,self).__init__()
    def forward(self,x):
        return x-x.mean()
layer = CenteredLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))
#也可以用它构造更复杂模型
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())
y = net(torch.rand(4,8))
y.mean().item()

5.587935447692871e-09

In [5]:
#含参数的模型定义
#在自定义含模型参数的层时，我们应该将参数定义成Parameter，除了直接定义成Parameter类外，
#还可以使用ParameterList和ParameterDict分别定义参数的列表和字典。
#ParameterList接收一个Parameter实例的列表作为输入然后得到一个参数列表，
#使用的时候可以用索引来访问某个参数，另外也可以使用append和extend在列表后面新增参数。
class MyDense(nn.Module):
    def __init__(self):
        super(MyDense,self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))
    def forward(self,x):
        for i in range(len(self.params)):
            x = torch.mm(x,self.params[i])
        return x
net = MyDense()
x = torch.rand(4,4)
print(net)
print(net(x))

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)
tensor([[14.4746],
        [16.2503],
        [ 6.8214],
        [ 5.5919]], grad_fn=<MmBackward>)


In [8]:
#ParameterDict接收一个Parameter实例的字典作为输入然后得到一个参数字典
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params = nn.ParameterDict({
            'linear1':nn.Parameter(torch.randn(4,4)),
            'linear2':nn.Parameter(torch.randn(4,1))
        })
        #使用update新增参数
        self.params.update({'linear3':nn.Parameter(torch.randn(4,2))})
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.params[choice])
net = MyDictDense()
print(net)
x = torch.ones(1,4)
print(net(x,'linear1'))
#自定义模型也可以用Squential类叠加

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)
tensor([[ 2.2242,  0.4691, -0.3615,  2.9520]], grad_fn=<MmBackward>)
